In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.corpus import words
from spellchecker import SpellChecker

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

from nltk.stem import WordNetLemmatizer

In [2]:
train = pd.read_csv('tweet_NLP_train.csv', encoding="ISO-8859-1")
test = pd.read_csv('tweet_NLP_test.csv', encoding="ISO-8859-1")
df = train.append(test, sort = False)

In [3]:
df.head(20)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [4]:
df['Sentiment'].unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

In [5]:
mapping = {'Neutral' : 0, 'Positive' : 1, 'Extremely Negative' : -1, 'Negative' : -1,'Extremely Positive' : 1}
df['label'] = df['Sentiment'].map(mapping)

In [6]:
df.head(20)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,label
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,0
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,1
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,1
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,1
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,-1
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive,1
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive,1
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral,0
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive,1
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative,-1


In [7]:
columns_to_keep = ['OriginalTweet','label']
df = df[columns_to_keep]

In [8]:
df.head(20)

,OriginalTweet,label
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,0
1,advice Talk to your neighbours family to excha...,1
2,Coronavirus Australia: Woolworths to give elde...,1
3,My food stock is not the only one which is emp...,1
4,"Me, ready to go at supermarket during the #COV...",-1
5,As news of the regionÂs first confirmed COVID...,1
6,Cashier at grocery store was sharing his insig...,1
7,Was at the supermarket today. Didn't buy toile...,0
8,Due to COVID-19 our retail store and classroom...,1
9,"For corona prevention,we should stop to buy th...",-1


In [9]:
df.dropna(inplace=True)

In [10]:
import re
def text_cleaning_1(tweet):
    tweet = re.sub(r" usa ", " America ", tweet)
    tweet = re.sub(r" USA ", " America ", tweet)
    tweet = re.sub(r" u s ", " America ", tweet)
    tweet = re.sub(r" uk ", " England ", tweet)
    tweet = re.sub(r" UK ", " England ", tweet)
    tweet = re.sub(r"USAgov", "USA government", tweet)
    tweet = re.sub(r"the US", "America", tweet)
    tweet = re.sub(r"Coronavirus", " covid ", tweet)
    tweet = re.sub(r"Covid19", " covid ", tweet)
    tweet = re.sub(r"\W", " ", tweet)
    tweet = re.sub(r"_", " ", tweet)
    return str(tweet)

In [11]:
df['OriginalTweet'] = df['OriginalTweet'].apply(text_cleaning_1)
display(df['OriginalTweet'].head(5))

0     MeNyrbie  Phil Gahan  Chrisitv https   t co i...
1    advice Talk to your neighbours family to excha...
2     covid  Australia  Woolworths to give elderly ...
3    My food stock is not the only one which is emp...
4    Me  ready to go at supermarket during the  COV...
Name: OriginalTweet, dtype: object

### Now remove url

In [12]:
def url_cleaning(tweet):
    tweet = re.sub('http://\S+|https://\S+',' ', tweet)
    
df['OriginalTweet'] = df['OriginalTweet'].apply(text_cleaning_1)
display(df['OriginalTweet'].head(5))

0     MeNyrbie  Phil Gahan  Chrisitv https   t co i...
1    advice Talk to your neighbours family to excha...
2     covid  Australia  Woolworths to give elderly ...
3    My food stock is not the only one which is emp...
4    Me  ready to go at supermarket during the  COV...
Name: OriginalTweet, dtype: object

In [13]:
df.head(20)

,OriginalTweet,label
0,MeNyrbie Phil Gahan Chrisitv https t co i...,0
1,advice Talk to your neighbours family to excha...,1
2,covid Australia Woolworths to give elderly ...,1
3,My food stock is not the only one which is emp...,1
4,Me ready to go at supermarket during the COV...,-1
5,As news of the regionÂ s first confirmed COVID...,1
6,Cashier at grocery store was sharing his insig...,1
7,Was at the supermarket today Didn t buy toile...,0
8,Due to COVID 19 our retail store and classroom...,1
9,For corona prevention we should stop to buy th...,-1


### Lower Case

In [14]:
df['OriginalTweet'] = df['OriginalTweet'].str.lower()

### Removing Stop words

In [15]:
def stop_word(tweet): 
    stop_words = set(stopwords.words('english')) 

    word_tokens = word_tokenize(tweet) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 

    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    return ' '.join(filtered_sentence)

In [16]:
df['OriginalTweet'] = df['OriginalTweet'].apply(stop_word)

In [17]:
df['OriginalTweet'].head(20)

0     menyrbie phil gahan chrisitv https co ifz9fan2...
1     advice talk neighbours family exchange phone n...
2     covid australia woolworths give elderly disabl...
3     food stock one empty please panic enough food ...
4     ready go supermarket covid19 outbreak paranoid...
5     news regionâ first confirmed covid 19 case cam...
6     cashier grocery store sharing insights covid 1...
7     supermarket today buy toilet paper rebel toile...
8     due covid 19 retail store classroom atlanta op...
9     corona prevention stop buy things cash use onl...
10    month crowding supermarkets restaurants howeve...
11    due covid 19 situation increased demand food p...
12    horningsea caring community letâ look less cap...
13    need stock food amazon deliver whatever need c...
14    adara releases covid 19 resource center travel...
15    lines grocery store unpredictable eating safe ...
16                               13 https co 51bl8p6vzh
17    eyeonthearctic 16mar20 russia consumer sur

In [18]:
spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    words = text.split()
    for word in words:
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        elif word not in misspelled_words:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [19]:
#df['OriginalTweet'] = df['OriginalTweet'].apply(correct_spellings)

### Tokenize and Lemmatizer

In [20]:
df['OriginalTweet'] = df['OriginalTweet'].apply(word_tokenize)
lem = WordNetLemmatizer()
def lemma_wordnet(input):
    return [lem.lemmatize(w) for w in input]
df['OriginalTweet'] = df['OriginalTweet'].apply(lemma_wordnet)

In [21]:
def combine_word(tweet):
    return " ".join(tweet)
df['OriginalTweet'] = df['OriginalTweet'].apply(combine_word)

#### Split data into Train and Test data sets 

In [22]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df['OriginalTweet'], 
                                                    df['label'], 
                                                    random_state=0)

## Vectorization with CountVectorizer

In [23]:
vect = CountVectorizer(min_df=5, ngram_range=[1,4], analyzer='char_wb').fit(X_train)
X_train_vect = vect.transform(X_train)
X_test_vect = vect.transform(X_test)

## Vectorization with TFID vectorizer

In [24]:
vect = TfidfVectorizer(min_df=3, ngram_range=[1,4]).fit(X_train)
X_train_vect_TFID = vect.transform(X_train)
X_test_vect_TFID = vect.transform(X_test)

## Applying ML

In [25]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

#### MultinomialNB

In [26]:
print("MultinomialNB with CountVectorizer\n")
alpha = [0.01, 0.1, 1.0, 5.0, 10.0, 20.0, 100.0]
for value in alpha:
    model = MultinomialNB(alpha = value).fit(X_train_vect, y_train)
    y_predicted = model.predict(X_test_vect)
    score = multiclass_roc_auc_score(y_test, y_predicted)
    acc_score = accuracy_score(y_test, y_predicted)
    print(f"With alpha set to {value}, AUC score of model is {score} and Accuracy Score is {acc_score}\n")

MultinomialNB with CountVectorizer

With alpha set to 0.01, AUC score of model is 0.7076762305448505 and Accuracy Score is 0.6181154907020198

With alpha set to 0.1, AUC score of model is 0.7089731861739362 and Accuracy Score is 0.615001334638313

With alpha set to 1.0, AUC score of model is 0.7085905764124826 and Accuracy Score is 0.6152682623009165

With alpha set to 5.0, AUC score of model is 0.7047579453402403 and Accuracy Score is 0.6341311504582258

With alpha set to 10.0, AUC score of model is 0.6837234774655566 and Accuracy Score is 0.6345760298958982

With alpha set to 20.0, AUC score of model is 0.6411217623851059 and Accuracy Score is 0.6080612154106237

With alpha set to 100.0, AUC score of model is 0.5318652464928841 and Accuracy Score is 0.4720170833704066



In [27]:
print("MultinomialNB with Tfid Vectorizer\n")
alpha = [0.01, 0.1, 1.0, 5.0, 10.0, 20.0, 100.0]
for value in alpha:
    model = MultinomialNB(alpha = value).fit(X_train_vect_TFID, y_train)
    y_predicted = model.predict(X_test_vect_TFID)
    score = multiclass_roc_auc_score(y_test, y_predicted)
    acc_score = accuracy_score(y_test, y_predicted)
    print(f"With alpha set to {value}, AUC score of model is {score} and Accuracy Score is {acc_score}\n")

MultinomialNB with Tfid Vectorizer

With alpha set to 0.01, AUC score of model is 0.7016157211282003 and Accuracy Score is 0.6507696414271732

With alpha set to 0.1, AUC score of model is 0.7077899912964001 and Accuracy Score is 0.6615357238188451

With alpha set to 1.0, AUC score of model is 0.6650680543357416 and Accuracy Score is 0.6404484384731738

With alpha set to 5.0, AUC score of model is 0.6241530219556921 and Accuracy Score is 0.5966723018062106

With alpha set to 10.0, AUC score of model is 0.6013522916955812 and Accuracy Score is 0.5666874277070914

With alpha set to 20.0, AUC score of model is 0.5769078591626444 and Accuracy Score is 0.5336773734318

With alpha set to 100.0, AUC score of model is 0.524065650984248 and Accuracy Score is 0.46169588041640713



####  Decision Tree Classifier

In [28]:
print("DecisionTreeClassifier with CountVectorizer\n")
depth = [3,6,9,12,15]
for value in depth:
    model = DecisionTreeClassifier(max_depth = value).fit(X_train_vect, y_train)
    y_predicted = model.predict(X_test_vect)
    score = multiclass_roc_auc_score(y_test, y_predicted)
    acc_score = accuracy_score(y_test, y_predicted)
    print(f"With max_depth set to {value}, AUC score of model is {score} and Accuracy Score is {acc_score}\n")

DecisionTreeClassifier with CountVectorizer

With max_depth set to 3, AUC score of model is 0.5487759083521428 and Accuracy Score is 0.49310436871607793

With max_depth set to 6, AUC score of model is 0.6205902679629031 and Accuracy Score is 0.5150814129370941

With max_depth set to 9, AUC score of model is 0.652661831247629 and Accuracy Score is 0.5446214075985408

With max_depth set to 12, AUC score of model is 0.6591147723266861 and Accuracy Score is 0.5694456802206602

With max_depth set to 15, AUC score of model is 0.658547908797754 and Accuracy Score is 0.5746062816976599



In [29]:
print("DecisionTreeClassifier with CountVectorizer\n")
depth = [3,6,9,12,15]
for value in depth:
    model = DecisionTreeClassifier(max_depth = value).fit(X_train_vect_TFID, y_train)
    y_predicted = model.predict(X_test_vect_TFID)
    score = multiclass_roc_auc_score(y_test, y_predicted)
    acc_score = accuracy_score(y_test, y_predicted)
    print(f"With max_depth set to {value}, AUC score of model is {score} and Accuracy Score is {acc_score}\n")

DecisionTreeClassifier with CountVectorizer

With max_depth set to 3, AUC score of model is 0.5460235890751676 and Accuracy Score is 0.49061304386511256

With max_depth set to 6, AUC score of model is 0.5468621456135127 and Accuracy Score is 0.4912358750778539

With max_depth set to 9, AUC score of model is 0.6307649126532987 and Accuracy Score is 0.5172168342379215

With max_depth set to 12, AUC score of model is 0.6450699896755275 and Accuracy Score is 0.5400836373342824

With max_depth set to 15, AUC score of model is 0.6513280954939291 and Accuracy Score is 0.5505827920633508



### Logistic Regression

In [30]:
print("Logistic Regression with CountVectorizer\n")
C = [ 1, 5, 10, 20, 100, 1000]
for value in C:
    model = LogisticRegression(C = value,solver='lbfgs').fit(X_train_vect, y_train)
    y_predicted = model.predict(X_test_vect)
    score = multiclass_roc_auc_score(y_test, y_predicted)
    acc_score = accuracy_score(y_test, y_predicted)
    print(f"With C set to {value}, AUC score of model is {score} and Accuracy Score is {acc_score}\n")

Logistic Regression with CountVectorizer



/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


With C set to 1, AUC score of model is 0.7570491311087567 and Accuracy Score is 0.7029984874099119



/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


With C set to 5, AUC score of model is 0.7550202650163224 and Accuracy Score is 0.698905596583326



...

/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


With C set to 20, AUC score of model is 0.7558165809989101 and Accuracy Score is 0.7021087285345672



/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


With C set to 100, AUC score of model is 0.7634893512324838 and Accuracy Score is 0.7102945101877391

With C set to 1000, AUC score of model is 0.76162750944433 and Accuracy Score is 0.70673547468636



/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [31]:
print("Logistic Regression with Tfid Vectorizer\n")
C = [ 1, 5, 10, 20, 100, 1000]
for value in C:
    model = LogisticRegression(C = value,solver='lbfgs').fit(X_train_vect_TFID, y_train);
    y_predicted = model.predict(X_test_vect_TFID); 
    score = multiclass_roc_auc_score(y_test, y_predicted);
    acc_score = accuracy_score(y_test, y_predicted)
    print(f"With C set to {value}, AUC score of model is {score} and Accuracy Score is {acc_score}\n");

Logistic Regression with Tfid Vectorizer



/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


With C set to 1, AUC score of model is 0.799893323786355 and Accuracy Score is 0.7685737165228224



/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


With C set to 5, AUC score of model is 0.8165073331407777 and Accuracy Score is 0.7814752202153217



...

/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


With C set to 20, AUC score of model is 0.8204510716127781 and Accuracy Score is 0.7828988344158733



/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


With C set to 100, AUC score of model is 0.8167457630049272 and Accuracy Score is 0.7774713052762702

With C set to 1000, AUC score of model is 0.8080927631948708 and Accuracy Score is 0.7668831746596673



/home/dipeshkusrai/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### Rigid

In [32]:
print("Rigid with Tfid Vectorizer\n")
C = [ 1, 5, 10, 20, 100, 1000]
for value in C:
    model = RidgeClassifier(alpha = value).fit(X_train_vect_TFID, y_train)
    y_predicted = model.predict(X_test_vect_TFID)
    score = accuracy_score(y_test, y_predicted)
    acc_score = accuracy_score(y_test, y_predicted)
    print(f"With C set to {value}, AUC score of model is {score} and Accuracy Score is {acc_score}\n");

Rigid with Tfid Vectorizer

With C set to 1, AUC score of model is 0.7652816086840466 and Accuracy Score is 0.7652816086840466

With C set to 5, AUC score of model is 0.7545155262923747 and Accuracy Score is 0.7545155262923747

With C set to 10, AUC score of model is 0.7303140848829968 and Accuracy Score is 0.7303140848829968

With C set to 20, AUC score of model is 0.6971260788326363 and Accuracy Score is 0.6971260788326363

With C set to 100, AUC score of model is 0.6028116380460895 and Accuracy Score is 0.6028116380460895

With C set to 1000, AUC score of model is 0.43527004181866713 and Accuracy Score is 0.43527004181866713

